## Lab. 12

### Solve the following problem using Genetic Algorithms:


Problem: Weighted N-Queen Problem


You are given an N×N chessboard, and each cell of the board has an associated weight. Your task is to find a valid placement of N queens such that the total weight of the queens is maximized, and no two queens threaten each other.





In the traditional N-Queen Problem, the goal is to place N queens on an N×N chessboard in such a way that no two queens threaten each other. In this variation, we introduce weights to the queens and aim to find a placement that maximizes the total weight of the queens while satisfying the constraint of non-threatening positions.


Constraints:

1. There should be exactly one queen in each row and each column.
2. No two queens should be placed in the same diagonal, i.e., they should not threaten each other.
3. The placement should maximize the total weight of the queens.


Representation:

Use a permutation-based representation. Each permutation represents the column position of the queen for each row. 

For example, if N=4, a valid permutation [2, 4, 1, 3] indicates that the queen in the first row is placed in column 2, the queen in the second row is placed in column 4, and so on.


Genetic Algorithm Steps:

1. *Initialization*: Generate an initial population of permutations randomly.

2. *Fitness Evaluation*: Evaluate the fitness of each permutation by calculating the total weight of the queens while considering the non-threatening positions.

3. *Selection*: Select a subset of permutations from the population based on their fitness, using selection techniques like tournament selection or roulette wheel selection.

4. *Crossover*: Perform crossover (recombination) on the selected permutations to create new offspring permutations.

5. *Mutation*: Introduce random changes (mutations) in the offspring permutations to maintain diversity in the population.

6. *Fitness Evaluation for the new individuals*: Evaluate the fitness of the new population.

7. *Form the new population*: Select the surviving individuals based on scores, with chances direct proportional with their performance.

8. Repeat steps 3-7 for a certain number of generations or until a termination condition is met (e.g., a maximum number of iterations or a satisfactory solution is found).


9. *Termination*: Return the best-performing individual (permutation) found as the solution to the problem.

Note: The fitness function used in this problem should calculate the total weight of the queens based on the positions specified by the permutation. Additionally, the fitness function should penalize solutions that violate the non-threatening constraint by assigning a lower fitness score to such permutations.

In [92]:
import random
import numpy as np

N = 8 #number of queens

def generate_weights(N):
    population = []
    for _ in range(N):
        individual = random.sample(range(N*N), N) 
        population.append(individual)
    return population
weights = generate_weights(N)
print("\nWeights:\n")
print(weights)

def initialize_population(population_size, N):
    population = []
    for _ in range(population_size):
        individual = random.sample(range(N), N)
        population.append(individual)
    return population

population = initialize_population(7, N)
print("\nPopulation:\n")
print(population)

def evaluate_fitness(population, weights, N):
    fitness_scores = []
    for individual in population:
        totalWeight = 0
    
        for row in range(0, N):
            queenColumn = individual[row]
            queenWeight = weights[row][queenColumn]

            for prevRow in range(0, row):
                prevQueenColumn = individual[prevRow]
                if queenColumn == prevQueenColumn or abs(queenColumn - prevQueenColumn) == abs(row - prevRow):
                    queenWeight = queenWeight * 0.5  # Penalize queens that threaten each other
        
            totalWeight += queenWeight
            
        fitness_scores.append(totalWeight)
    
    return fitness_scores

fitness_scores = evaluate_fitness(population, weights, N)
print("\nFitness:\n")
print(fitness_scores)

def selection(population, fitness_scores, numParents):
    total_fitness = sum(fitness_scores)
    selection_probs = [fitness / total_fitness for fitness in fitness_scores]

    
    # Select two parents using roulette wheel selection
    parents = []
    for _ in range(numParents):
        r = random.random()  # Generate a random number between 0 and 1
        cumulative_prob = 0
        for i, individual in enumerate(population):
            cumulative_prob += selection_probs[i]
            if r <= cumulative_prob:
                parents.append(individual)
                break

    return parents


parents = selection(population,fitness_scores, 6)
print("\nParents:\n")
print(parents)

#4 Crossover

def crossover(parents):
    point1 = random.randint(0, len(parents[0]) // 2 - 1)
    point2 = random.randint(len(parents[0]) // 2, len(parents[0]) - 1)
    parent1 = parents[0]
    parent2 = parents[1]
    parent1_chromosomes = parent1[point1:point2]
    
    child = []
    count = 0
    x = 0
    
    while count < point1:
        if parent2[x] not in parent1_chromosomes:
            child.append(parent2[x])
            count = count + 1
        x = x + 1
    for i in range(point1, point2 + 1):
        child.append(parent1[i])
        count = count + 1
    while count < len(parent2):
        if parent2[x] not in child:
            child.append(parent2[x])
            count = count + 1
        x = x + 1

    return child

offspring = []

for i in range(0, len(parents), 2):
    parent1 = parents[i]
    parent2 = parents[i + 1]
    newParents = [parent1, parent2]
    offspring.append(crossover(newParents))
    
print("\nCrossOver:\n")
print(offspring)


# Applies mutation to each child in the offspring list with a probability of 0.1
def mutate(child, mutation_rate):
    val = random.uniform(0.0, 1.0)
    if mutation_rate > val:
        pos_1 = random.randint(0, len(child) // 2)
        pos_2 = random.randint(0, len(child) // 2)
        child[pos_1], child[pos_2] = child[pos_2], child[pos_1]
    return child

mutated = [mutate(child, 0.1) for child in offspring]
print("\nMutated:\n")
print(mutated)

#6 Fitness Evaluation
# Calculates the fitness value for each child in the offspring
child_scores = evaluate_fitness(mutated, weights, N)

print("\nChild Fitness:\n")
print(child_scores)

#7 Form the new population
# Forms a new population by selecting individuals from the offspring based on their fitness scores
def form_new_population(population, scores, population_size):
    selection_probabilities = scores / np.sum(scores)
    new_population_indices = np.random.choice(len(population), size=population_size, p=selection_probabilities)
    new_population = [population[i] for i in new_population_indices]
    return new_population

new_population = form_new_population(mutated, child_scores, N)
print("\nNew population:\n")
print(new_population)


def genetic_algorithm(population_size, num_generations, mutation_rate, N):
    weights = generate_weights(N)
    population = initialize_population(population_size, N)
    #print(len(population))
    
    for _ in range(num_generations):
        # Fitness evaluation
        fitness_scores = evaluate_fitness(population, weights, N)
        print(len(fitness_scores))
        # Selection
        parentss = selection(population,fitness_scores, 20)
        

        # Crossover
        offspring = []
        for i in range(0, len(parentss), 2):
            #print(i)
            parent1 = parentss[i]
            parent2 = parentss[i + 1]
            parents = [parent1, parent2]
            #print(parents)
            offspring.append(crossover(parents))
            
        # Mutated
        mutations = [mutate(child, mutation_rate) for child in offspring]
        mutations_fitness_values = evaluate_fitness(mutations, weights, N)
        population = form_new_population(mutations, mutations_fitness_values, N)

    return population

population = genetic_algorithm(100, 1, 0.1, 8)
print("Generations:", 100)
fitness_values = evaluate_fitness(population, weights, N)
best_index = np.argmax(fitness_values)
best_individual = population[best_index]
print("Best Individual:", best_individual)


Weights:

[[30, 20, 44, 62, 61, 28, 52, 35], [1, 10, 58, 46, 44, 25, 14, 21], [34, 46, 55, 49, 40, 43, 20, 19], [18, 27, 20, 6, 12, 33, 63, 23], [24, 16, 42, 28, 50, 35, 60, 45], [45, 62, 10, 24, 7, 51, 49, 41], [1, 54, 10, 52, 29, 31, 0, 35], [30, 36, 25, 6, 38, 51, 50, 37]]

Population:

[[4, 7, 3, 1, 5, 0, 2, 6], [4, 0, 1, 5, 3, 6, 2, 7], [3, 5, 0, 6, 7, 1, 2, 4], [0, 7, 3, 4, 1, 6, 5, 2], [3, 7, 0, 5, 4, 1, 2, 6], [0, 6, 5, 3, 1, 4, 7, 2], [3, 1, 0, 4, 5, 2, 6, 7]]

Fitness:

[239.5, 216.0, 218.75, 166.75, 258.625, 139.0, 132.75]

Parents:

[[4, 7, 3, 1, 5, 0, 2, 6], [3, 7, 0, 5, 4, 1, 2, 6], [0, 7, 3, 4, 1, 6, 5, 2], [4, 7, 3, 1, 5, 0, 2, 6], [0, 7, 3, 4, 1, 6, 5, 2], [4, 0, 1, 5, 3, 6, 2, 7]]

CrossOver:

[[3, 7, 0, 1, 5, 4, 2, 6], [0, 7, 3, 4, 1, 6, 5, 2], [0, 5, 3, 4, 1, 6, 5, 2]]

Mutated:

[[3, 7, 0, 1, 5, 4, 2, 6], [0, 7, 3, 4, 1, 6, 5, 2], [0, 5, 3, 4, 1, 6, 5, 2]]

Child Fitness:

[199.0, 166.75, 163.0]

New population:

[[0, 7, 3, 4, 1, 6, 5, 2], [0, 5, 3, 4, 1, 6, 5, 2]